Send Feedback
Titanic dataset is one of the datasets available in sklearn.
You are given:
    1. A Training dataset csv file with X train and Y train data
    2. A X test File and you have to predict and submit predictions for this file.
Your task is to:
    1. Use Logistic Regression and come with predictions.
Read Instructions carefully -
    1. Use Logistic Regression as a training algorithm and submit results predicted.
    2. Files are in csv format.
    3. Submit a csv file with only predictions for X test data. File should not have any headers and should only have one column i.e. predictions.
    4. Your score is based on number of accurate predictions.

In [1]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA


In [2]:
# Loading Dataset
dataframe = pd.read_csv("training_titanic_x_y_train.csv", delimiter = ",")


df = dataframe.copy()
sex = df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(1 if s == 'male' else 0)
df['Sex'] = sex_int
df.drop('Name', axis = 1, inplace = True)
df.drop('Ticket', axis = 1, inplace = True)
df.drop('Cabin', axis = 1, inplace = True)
df.drop('Embarked', axis = 1, inplace = True)
ages = df['Age'].fillna(0)
mean_age = np.array(ages).mean()
print(mean_age)
df.fillna(mean_age, inplace = True)
print(df)
df.describe()

X_df = df.drop('Survived', axis = 1)
X_train = X_df.values
Y_train = df['Survived'].values

print("Shape of X_train ", X_train.shape)
print("Shape of Y_train ", Y_train.shape)

test_dataframe = pd.read_csv("test_titanic_x_test.csv", delimiter = ",")


test_df = test_dataframe.copy()
sex = test_df['Sex'].values
sex_int = []
for s in sex :
    sex_int.append(1 if s == 'male' else 0)
test_df['Sex'] = sex_int
test_df.drop('Name', axis = 1, inplace = True)
test_df.drop('Ticket', axis = 1, inplace = True)
test_df.drop('Cabin', axis = 1, inplace = True)
test_df.drop('Embarked', axis = 1, inplace = True)
ages = test_df['Age'].fillna(0)
mean_age = np.array(ages).mean()
print(mean_age)
test_df.fillna(mean_age, inplace = True)
print(test_df)
test_df.describe()

X_test = test_df.values

print("Shape of X_test ", X_test.shape)


23.8315868263
     Pclass  Sex        Age  SibSp  Parch      Fare  Survived
0         2    0  29.000000      1      0   26.0000         1
1         3    1  23.831587      0      0    8.0500         0
2         2    1  39.000000      0      0   26.0000         0
3         3    0  29.000000      0      4   21.0750         0
4         3    1  25.000000      0      0    7.0500         0
5         3    1  34.500000      0      0    6.4375         0
6         1    0  35.000000      1      0   53.1000         1
7         3    1  23.000000      0      0    7.8958         0
8         2    1  23.831587      0      0   13.0000         1
9         3    1  50.000000      0      0    8.0500         0
10        3    0  23.000000      0      0    7.5500         1
11        3    1  23.831587      0      0    7.2292         0
12        2    1  36.000000      0      0   13.0000         0
13        3    1  23.831587      1      0   15.5000         0
14        3    0  23.831587      0      0    7.7500     

In [3]:
model = LogisticRegression(tol = 0.000001, solver = 'saga', max_iter = 10000000, C = 0.1, warm_start = True)
model.fit(X_train, Y_train)

#Y_pred = model.predict(X_test)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
def train_n_models(X_train, Y_train, X_test):
    n = 10
    sub_Size = 1
    Y_pred = np.zeros(len(X_test))
    Y_train_pred = np.zeros(int(sub_Size * len(X_train)))
    num_s = 0
    for i in range(n):
        X1, X2, Y1, Y2 = model_selection.train_test_split(X_train, Y_train, test_size = 1 - sub_Size)
        #model = LogisticRegression(tol = 0.000001, solver = 'saga', max_iter = 10000000, C = 0.1, warm_start = True)
        model = GradientBoostingClassifier(loss = 'exponential', n_estimators = 1000, subsample = 1,  warm_start = True)      
        model.fit(X1, Y1)
        #print("Training Score : ", model.score(X_train, Y_train))
        score = model.score(X1, Y1)
        if score > 0:   
            print(num_s, " Training Score : ", score)
            Y_i_pred = model.predict(X_test)
            Y_pred += Y_i_pred
            Y_train_i_pred = model.predict(X1)
            Y_train_pred += Y_train_i_pred
            num_s += 1
    Y_pred /= num_s
    Y_train_pred /= num_s          
    
    #error = Y_train_pred - Y_pred
    
    pred = Y_pred.round()
    train_pred = Y_train_pred.round()
    
    #print(Y_pred, pred)
    return train_pred, pred

Y_train_pred, Y_pred = train_n_models(X_train, Y_train, X_test)



#print("Training Score : ", model.score(X_train, Y_train))

#Training Score :  0.784431137725 
#Training Score :  0.790419161677 (tol = 0.000001, solver = 'saga', max_iter = 10000000, C = 1, warm_start = True)
# (tol = 0.000001, solver = 'saga', max_iter = 10000000, C = 0.5, warm_start = True)


0  Training Score :  0.980538922156
1  Training Score :  0.980538922156
2  Training Score :  0.980538922156
3  Training Score :  0.980538922156
4  Training Score :  0.980538922156
5  Training Score :  0.980538922156
6  Training Score :  0.980538922156
7  Training Score :  0.980538922156
8  Training Score :  0.980538922156
9  Training Score :  0.980538922156


In [4]:
np.savetxt("y_pred.csv", Y_pred)

In [5]:
model.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000000, multi_class='ovr',
          n_jobs=1, penalty='l2', random_state=None, solver='saga',
          tol=1e-06, verbose=0, warm_start=True)>

In [8]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_train, Y_train_pred))

[[332  67]
 [220  49]]
